<a href="https://colab.research.google.com/github/vyom10445/toxicity-classifier/blob/main/notebooks/Toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
!git clone https://github.com/vyom10445/toxicity-classifier.git

Cloning into 'toxicity-classifier'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 41 (delta 11), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (41/41), 26.45 MiB | 13.84 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
import os
os.chdir("toxicity-classifier")

In [4]:
df = pd.read_csv(
    os.path.join(
        "data",
        "jigsaw-toxic-comment-classification-challenge",
        "train.csv"
    )
)

In [5]:
from tensorflow.keras.layers import TextVectorization #preprocessing

In [6]:
x = df['comment_text']
y = df[df.columns[2:]].values

In [7]:
MAX_FEATURES = 200000 # number of words in the vocab

In [8]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [9]:
vectorizer.adapt(x.values)

In [10]:
vectorized_text = vectorizer(x.values)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottlenecks

In [12]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [13]:
#build a sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dropout,Dense,Embedding

In [14]:
#instantiate our model(instantiate the sequential api)
model=Sequential()
#create the embedding layer
model.add(Embedding(MAX_FEATURES+1,32))
#bidirectional LSTM layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers(dense layers)
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#Final layer
model.add(Dense(6,activation='sigmoid')) #maps to the no. of different of outputs that we have got inside our neural network

In [15]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [16]:
history = model.fit(train , epochs=1 , validation_data=val)

6981/6981 ━━━━━━━━━━━━━━━━━━━━ 689s 98ms/step - loss: 0.0811 - val_loss: 0.0464
